What Factors Affect STEM Degree Completion Rates at Public Universities in the United States?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns


In [35]:
# Load the datasets
completions = pd.read_csv('IPEDS_Completions.csv')
institutions = pd.read_csv('IPEDS_IC.csv')
Directory = pd.read_csv('IPEDS_Directory.csv')
demographics = pd.read_csv('IPEDS_EFFY.csv')
Chareges = pd.read_csv('IPEDS_Charges.csv')

In [18]:
# shape of the loaded Datasets withe lable  as a table
print('completions shape:', completions.shape)
print('institutions shape:', institutions.shape)
print('Chareges shape:', Chareges.shape)




completions shape: (300877, 64)
institutions shape: (6138, 136)
Chareges shape: (3876, 235)


In [ ]:
completions.head()

In [ ]:
institutions.head()

In [36]:
#keep only the first 20 columns
institutions = institutions.iloc[:, :20]

In [37]:
# from institutions if the row have -ve value change to 0 else change it to 1 accross all columns except the first column
institutions.iloc[:, 1:] = np.where(institutions.iloc[:, 1:] < 0, 0, 1)

In [38]:
#store only distinct values in the institutions
institutions = institutions.drop_duplicates()

In [ ]:
#Directory.head()

In [ ]:
#demographics.head()

In [19]:
#filter only UNITID TUITION1 FEE1 HRCHG1
Chareges = Chareges[['UNITID', 'TUITION1', 'FEE1', 'HRCHG1']]


In [5]:
# filter only UNITID and CONTROL from directory
Directory = Directory[['UNITID', 'CONTROL']]

In [6]:
# Filter for public universities
public_universities =  Directory[Directory['CONTROL'] == 1]
public_university_ids = public_universities['UNITID'].unique()

In [7]:
# Define the CIP codes for STEM fields
# Replace with actual CIP codes for STEM fields
stem_cip_codes = [11, 14, 15, 26, 27, 40, 41, 52, 54, 62, 91, 15, 26, 52, 54, 62, 91]

In [8]:
# Filter completions for public universities and STEM fields CIP codes
completions['CIPCODE'] = completions['CIPCODE'].astype(str).str.split('.').str[0].astype(int)

In [9]:

completions_stem = completions[(completions['UNITID'].isin(public_university_ids)) & 
                               (completions['CIPCODE'].isin(stem_cip_codes))]
completions_stem.head()


,UNITID,CIPCODE,MAJORNUM,AWLEVEL,XCTOTALT,CTOTALT,XCTOTALM,CTOTALM,XCTOTALW,CTOTALW,...,XCUNKNM,CUNKNM,XCUNKNW,CUNKNW,XCNRALT,CNRALT,XCNRALM,CNRALM,XCNRALW,CNRALW
12,100654,11,1,5,R,41,R,23,R,18,...,R,1,R,2,Z,0,Z,0,Z,0
13,100654,11,1,7,R,16,R,9,R,7,...,R,1,R,1,Z,0,Z,0,Z,0
26,100654,14,1,5,R,7,R,3,R,4,...,Z,0,Z,0,R,1,R,1,Z,0
27,100654,14,1,5,R,18,R,16,R,2,...,Z,0,Z,0,R,2,R,2,Z,0
28,100654,14,1,5,R,30,R,21,R,9,...,R,1,R,0,Z,0,Z,0,Z,0


In [48]:
#remove all the columns that start with XC
completions_stem = completions_stem.loc[:,~completions_stem.columns.str.startswith('XC')]

In [52]:
#remove column with CIPCODE and aggrage the ohter columns except UNITID
completions_stem = completions_stem.groupby('UNITID').sum().reset_index()

In [ ]:
completions_stem['CIPCODE'].value_counts().reset_index().rename(columns={'index': 'CIPCODE', 'CIPCODE': 'count'})

In [54]:
#numerical columns in completions_stem standarize the data 
numerical_columns = completions_stem.columns[1:]
scaler = StandardScaler()
completions_stem[numerical_columns] = scaler.fit_transform(completions_stem[numerical_columns])


In [55]:
# Merge completions and charges data and isntitutions data
completions_charges = completions_stem.merge(Chareges, on='UNITID')
completions_charges_institutions = completions_charges.merge(institutions, on='UNITID')

In [41]:
completions_charges_institutions.head()


,UNITID,CIPCODE,MAJORNUM,AWLEVEL,CTOTALT,CTOTALM,CTOTALW,CAIANT,CAIANM,CAIANW,...,PUBSECON,RELAFFIL,LEVEL1,LEVEL1A,LEVEL1B,LEVEL2,LEVEL3,LEVEL4,LEVEL5,LEVEL6
0,100654,11,1,5,41,23,18,0,0,0,...,1,0,1,1,1,1,1,1,1,1
1,100654,11,1,7,16,9,7,0,0,0,...,1,0,1,1,1,1,1,1,1,1
2,100654,14,1,5,7,3,4,0,0,0,...,1,0,1,1,1,1,1,1,1,1
3,100654,14,1,5,18,16,2,0,0,0,...,1,0,1,1,1,1,1,1,1,1
4,100654,14,1,5,30,21,9,1,1,0,...,1,0,1,1,1,1,1,1,1,1


In [42]:
print('completions_charges_institutions shape:', completions_charges_institutions.shape)

completions_charges_institutions shape: (56296, 287)


In [43]:
completions_charges_institutions.describe()

,UNITID,CIPCODE,MAJORNUM,AWLEVEL,CTOTALT,CTOTALM,CTOTALW,CAIANT,CAIANM,CAIANW,...,PUBSECON,RELAFFIL,LEVEL1,LEVEL1A,LEVEL1B,LEVEL2,LEVEL3,LEVEL4,LEVEL5,LEVEL6
count,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,56296.000000,...,56296.000000,56296.0,56296.0,56296.0,56296.0,56296.0,56296.0,56296.0,56296.000000,56296.000000
mean,187716.307819,31.461294,1.054196,7.881892,20.454277,11.759024,8.695254,0.090468,0.044017,0.046451,...,0.999787,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999361,0.999361
std,69640.243464,17.503706,0.226405,6.664417,66.413833,40.222411,31.126684,0.626873,0.325949,0.418583,...,0.014599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025280,0.025280
min,100654.000000,11.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000
25%,141839.000000,14.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
50%,180179.000000,26.000000,1.000000,5.000000,5.000000,2.000000,2.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
75%,217819.000000,52.000000,1.000000,7.000000,16.000000,9.000000,6.000000,0.000000,0.000000,0.000000,...,1.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
max,498571.000000,54.000000,2.000000,21.000000,4554.000000,3735.000000,1471.000000,33.000000,23.000000,25.000000,...,1.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000


In [ ]:

# Feature Engineering
# Assuming 'INSTSIZE' and 'FUNDING' are available in the institutions dataset
features = completions_demographics[['YEAR', 'GENDER', 'RACE', 'AGE', 'INSTSIZE', 'FUNDING']]
target = completions_demographics['CIPCODE']


In [ ]:

# Handle categorical variables
categorical_features = ['GENDER', 'RACE']
numerical_features = ['YEAR', 'AGE', 'INSTSIZE', 'FUNDING']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Plot the results
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel('Actual Completions')
plt.ylabel('Predicted Completions')
plt.title('Actual vs Predicted Completions')
plt.show()

# Display feature coefficients
regressor = model.named_steps['regressor']
feature_names = preprocessor.transformers_[0][2] + list(preprocessor.transformers_[1][1].get_feature_names_out())
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': regressor.coef_
}).sort_values(by='Coefficient', ascending=False)

print(coefficients)